In [229]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import re
import string
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/quora-insincere-questions-classification/sample_submission.csv
/kaggle/input/quora-insincere-questions-classification/embeddings.zip
/kaggle/input/quora-insincere-questions-classification/train.csv
/kaggle/input/quora-insincere-questions-classification/test.csv
/kaggle/input/df-stack/df_stack.csv
/kaggle/input/df-stack/val_conf_matrix_fold_4.png
/kaggle/input/df-stack/val_conf_matrix_fold_2.png
/kaggle/input/df-stack/val_conf_matrix_fold_1.png
/kaggle/input/df-stack/val_conf_matrix_fold_3.png
/kaggle/input/df-stack/df_stack_test.csv
/kaggle/input/df-stack/val_conf_matrix_fold_0.png
/kaggle/input/h2oai-predict-the-llm/sample_submission.csv
/kaggle/input/h2oai-predict-the-llm/train.csv
/kaggle/input/h2oai-predict-the-llm/test.csv


In [230]:
train = pd.read_csv("/kaggle/input/h2oai-predict-the-llm/train.csv")
test = pd.read_csv("/kaggle/input/h2oai-predict-the-llm/test.csv")

In [231]:
trainResponseNull_idx = train[train.Response.isnull()].index
trainResponseNull_idx

Index([196, 339, 401, 798, 2279, 2725, 3046], dtype='int64')

In [232]:
train.Question.loc[trainResponseNull_idx]

196     Can you provide a guide to starting a meditati...
339     How do you manage to keep information concise ...
401     What safeguards are in place to prevent misuse...
798     How do you ensure the safety of the informatio...
2279    How can I improve my memory and recall abilities?
2725    Tires a horse, worries a man. Tell me this rid...
3046    What happens when a user asks you something ou...
Name: Question, dtype: object

In [233]:
testResponseNull_idx = test[test.Response.isnull()].index
testResponseNull_idx

Index([446, 969], dtype='int64')

In [234]:
train = train.fillna("")
test = test.fillna("")

In [235]:
!pip install -q mistune

In [236]:
from tqdm import tqdm
tqdm.pandas()

In [237]:
def build_vocab(sentences, verbose =  True):
    """
    :param sentences: list of list of words
    :return: dictionary of words and their count
    """
    vocab = {}
    for sentence in tqdm(sentences, disable = (not verbose)):
        for word in sentence:
            try:
                vocab[word] += 1
            except KeyError:
                vocab[word] = 1
    return vocab

In [262]:
sentences_q = train["Question"].progress_apply(lambda x: x.split()).values
sentences_r = train["Response"].progress_apply(lambda x: x.split()).values
sentences_q_test = test["Question"].progress_apply(lambda x: x.split()).values
sentences_r_test = test["Response"].progress_apply(lambda x: x.split()).values
vocab_q = build_vocab(sentences_q)
vocab_r = build_vocab(sentences_r)
vocab_q_test = build_vocab(sentences_q_test)
vocab_r_test = build_vocab(sentences_r_test)

print({k: vocab_q[k] for k in list(vocab_q)[:5]})

100%|██████████| 1001/1001 [00:00<00:00, 34387.14it/s]

{'Explain': 287, 'the': 3290, 'concept': 231, 'of': 2149, 'revolution': 7}


In [239]:
!unzip /kaggle/input/quora-insincere-questions-classification/embeddings.zip GoogleNews-vectors-negative300/

Archive:  /kaggle/input/quora-insincere-questions-classification/embeddings.zip


In [240]:
!rm -r paragram_300_sl999 wiki-news-300d-1M glove.840B.300d

rm: cannot remove 'paragram_300_sl999': No such file or directory
rm: cannot remove 'wiki-news-300d-1M': No such file or directory
rm: cannot remove 'glove.840B.300d': No such file or directory


In [263]:
from transformers import BertTokenizer, BertModel
import torch
import operator
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')
# Load the BERT tokenizer and model
# transformer_name = "distilroberta-base"

# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(transformer_name)



# from transformers import AutoModelForSequenceClassification

# model = AutoModelForSequenceClassification.from_pretrained(transformer_name, num_labels=7,ignore_mismatched_sizes=True
#                                                           )
def check_coverage(vocab, tokenizer, model):
    a = {}
    oov = {}
    k = 0
    i = 0

    for word in vocab:
        if word in tokenizer.vocab:
            k += vocab[word]
        else:
            oov[word] = vocab[word]
            i += vocab[word]

    print('Found embeddings for {:.2%} of vocab'.format(1 - len(oov) / len(vocab)))
    print('Found embeddings for {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

oov = check_coverage(vocab_q, tokenizer, model)
oov = check_coverage(vocab_r, tokenizer, model)
oov = check_coverage(vocab_q_test, tokenizer, model)
oov = check_coverage(vocab_r_test, tokenizer, model)

Found embeddings for 76.48% of vocab
Found embeddings for 81.28% of all text
Found embeddings for 42.90% of vocab
Found embeddings for 83.93% of all text
Found embeddings for 80.09% of vocab
Found embeddings for 82.24% of all text
Found embeddings for 56.53% of vocab
Found embeddings for 83.48% of all text


In [242]:
from gensim.models import KeyedVectors

news_path = '../input/embeddings/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin'
news_path = "/kaggle/working/GoogleNews-vectors-negative300/GoogleNews-vectors-negative300.bin"
embeddings_index = KeyedVectors.load_word2vec_format(news_path, binary=True)

import operator 

def check_coverage(vocab,embeddings_index):
    a = {}
    oov = {}
    k = 0
    i = 0
    for word in tqdm(vocab):
        try:
            a[word] = embeddings_index[word]
            k += vocab[word]
        except:

            oov[word] = vocab[word]
            i += vocab[word]
            pass

    print('Found embeddings for {:.2%} of vocab'.format(len(a) / len(vocab)))
    print('Found embeddings for  {:.2%} of all text'.format(k / (k + i)))
    sorted_x = sorted(oov.items(), key=operator.itemgetter(1))[::-1]

    return sorted_x

In [243]:
oov = check_coverage(vocab_q,embeddings_index)
oov = check_coverage(vocab_r,embeddings_index)
oov = check_coverage(vocab_q_test,embeddings_index)
oov = check_coverage(vocab_r_test,embeddings_index)

100%|██████████| 1840/1840 [00:00<00:00, 282114.32it/s]


Found embeddings for 64.18% of vocab
Found embeddings for  72.61% of all text


100%|██████████| 46011/46011 [00:00<00:00, 341317.74it/s]


Found embeddings for 40.96% of vocab
Found embeddings for  73.15% of all text


100%|██████████| 699/699 [00:00<00:00, 332556.54it/s]


Found embeddings for 69.10% of vocab
Found embeddings for  71.55% of all text


100%|██████████| 19279/19279 [00:00<00:00, 400031.60it/s]

Found embeddings for 49.58% of vocab
Found embeddings for  73.04% of all text


|process| train q | train r | test q |test r|
|-------|---------|---------|--------|------|
|no process|64.18% - 72.61% | 40.96% - 73.15% | 69.10% - 71.55% | 49.58% - 73.04%|

In [244]:
from bs4 import BeautifulSoup
import mistune
import re
import emoji
def clean_markdown_and_emojis(markdown_text):
    # Create a Mistune Markdown parser
#     markdown_parser = mistune.Markdown()
#     # Parse the Markdown text and convert it to plain text
#     html_text = markdown_parser(markdown_text)

    # Create a BeautifulSoup parser
    soup = BeautifulSoup(markdown_text, 'html.parser')
    # Extract plain text
    plain_text = soup.get_text()
    
    cleaned_text = re.sub(r"\r?\n", ' ', plain_text)
    cleaned_text = re.sub(r"_", ' ', cleaned_text)
    cleaned_text = re.sub(r"#", ' ', cleaned_text)
    cleaned_text = re.sub(r"\[", ' ', cleaned_text)  # Square brackets must be escaped with a backslash
    cleaned_text = re.sub(r"\]", ' ', cleaned_text)  # Square brackets must be escaped with a backslash
    cleaned_text = re.sub(r"\(", ' ', cleaned_text)  # Parentheses must be escaped with a backslash
    cleaned_text = re.sub(r"\)", ' ', cleaned_text)  # Parentheses must be escaped with a backslash
    cleaned_text = re.sub(r"\*", ' ', cleaned_text)  # Asterisk must be escaped with a backslash
    cleaned_text = re.sub(r'\d+\.\s*', ' ', cleaned_text)
    cleaned_text = re.sub(r'!\[\]\(http://\)', ' ', cleaned_text)
    # Remove "a. " style numbering
    cleaned_text = re.sub(r'\b[a-z]\.\s*', ' ', cleaned_text)
    cleaned_text = re.sub(r'\d+', ' ', cleaned_text)
    cleaned_text = re.sub(r'https?://\S+|www\.\S+|\S+@\S+', ' ', cleaned_text)

    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)
    cleaned_text = emoji_pattern.sub(r'', cleaned_text)

    return cleaned_text

In [245]:
train.Question = train.Question.apply(clean_markdown_and_emojis)
train.Response = train.Response.apply(clean_markdown_and_emojis)
test.Question = test.Question.apply(clean_markdown_and_emojis)
test.Response = test.Response.apply(clean_markdown_and_emojis)

/tmp/ipykernel_32/2510498917.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(markdown_text, 'html.parser')
/tmp/ipykernel_32/2510498917.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(markdown_text, 'html.parser')
/tmp/ipykernel_32/2510498917.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(markdown_text, 'html.parser')
/tmp/ipykernel_32/2510498917.py:12: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(markdown_text, 'html.parser')


In [246]:
oov = check_coverage(  build_vocab(train.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(train.Response.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Response.apply(lambda x: x.split())) ,embeddings_index)

100%|██████████| 1763/1763 [00:00<00:00, 381005.67it/s]


Found embeddings for 66.25% of vocab
Found embeddings for  72.99% of all text


100%|██████████| 39984/39984 [00:00<00:00, 355486.56it/s]


Found embeddings for 48.55% of vocab
Found embeddings for  74.01% of all text


100%|██████████| 681/681 [00:00<00:00, 245261.98it/s]


Found embeddings for 70.34% of vocab
Found embeddings for  71.69% of all text


100%|██████████| 17518/17518 [00:00<00:00, 387191.69it/s]

Found embeddings for 56.23% of vocab
Found embeddings for  73.95% of all text


|process| train q | train r | test q |test r|
|-------|---------|---------|--------|------|
| no process|64.18% - 72.61% | 40.96% - 73.15% | 69.10% - 71.55% | 49.58% - 73.04%|
| clean_markdown_and_emojis |66.25% - 72.99% | 48.55% - 74.01% | 70.34% - 71.69% | 56.23% - 73.95%|
| clean_markdown_and_emojis |66.25% - 72.99% | 48.55% - 74.01% | 70.34% - 71.69% | 56.23% - 73.95%|

In [247]:
def clean_text(x):

    x = str(x)
    for punct in "/-'":
        x = x.replace(punct, ' ')
    for punct in '&':
        x = x.replace(punct, f' {punct} ')
    for punct in '?!.,"#$%\'()*+-/:;<=>@[\\]^_`{|}~' + '“”’':
        x = x.replace(punct, '')
#     for punct in '?!.,"#$%\'():;<>@[\\]^_`{|}~' + '“”’':
#         x = x.replace(punct, '')
    return x

In [248]:
train.Question = train.Question.apply(clean_text)
test.Question = test.Question.apply(clean_text)
train.Response = train.Response.apply(clean_text)
test.Response = test.Response.apply(clean_text)

In [249]:
oov = check_coverage(  build_vocab(train.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(train.Response.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Response.apply(lambda x: x.split())) ,embeddings_index)

100%|██████████| 1553/1553 [00:00<00:00, 370626.12it/s]


Found embeddings for 99.03% of vocab
Found embeddings for  87.10% of all text


100%|██████████| 23463/23463 [00:00<00:00, 334327.44it/s]


Found embeddings for 95.97% of vocab
Found embeddings for  88.13% of all text


100%|██████████| 639/639 [00:00<00:00, 363411.56it/s]


Found embeddings for 98.28% of vocab
Found embeddings for  86.12% of all text


100%|██████████| 11611/11611 [00:00<00:00, 376597.36it/s]


Found embeddings for 97.89% of vocab
Found embeddings for  87.81% of all text


|preprocess_fn| train q | train r | test q |test r|
|-------|---------|---------|--------|------|
|  |vocab  - all text | vocab  - all text | vocab  - all text| vocab  - all text|
| no fn|64.18% - 72.61% | 40.96% - 73.15% | 69.10% - 71.55% | 49.58% - 73.04%|
| clean_markdown_and_emojis |66.25% - 72.99% | 48.55% - 74.01% | 70.34% - 71.69% | 56.23% - 73.95%|
| clean_text |99.03% - 87.10% | 95.97% - 88.13% | 98.28% - 86.12% | 97.89% - 87.81%|

In [250]:
import re

def clean_numbers(x):

    x = re.sub('[0-9]{5,}', '#####', x)
    x = re.sub('[0-9]{4}', '####', x)
    x = re.sub('[0-9]{3}', '###', x)
    x = re.sub('[0-9]{2}', '##', x)
    return x

In [251]:
# train.Question = train.Question.apply(clean_numbers)
# test.Question = test.Question.apply(clean_numbers)
# train.Response = train.Response.apply(clean_numbers)
# test.Response = test.Response.apply(clean_numbers)

In [252]:
oov = check_coverage(  build_vocab(train.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(train.Response.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Response.apply(lambda x: x.split())) ,embeddings_index)

100%|██████████| 1553/1553 [00:00<00:00, 358647.40it/s]


Found embeddings for 99.03% of vocab
Found embeddings for  87.10% of all text


100%|██████████| 23463/23463 [00:00<00:00, 373341.51it/s]


Found embeddings for 95.97% of vocab
Found embeddings for  88.13% of all text


100%|██████████| 639/639 [00:00<00:00, 232309.98it/s]


Found embeddings for 98.28% of vocab
Found embeddings for  86.12% of all text


100%|██████████| 11611/11611 [00:00<00:00, 405610.78it/s]

Found embeddings for 97.89% of vocab
Found embeddings for  87.81% of all text


|preprocess_fn| train q | train r | test q |test r|
|-------|---------|---------|--------|------|
|  |vocab  - all text | vocab  - all text | vocab  - all text| vocab  - all text|
| no fn|64.18% - 72.61% | 40.96% - 73.15% | 69.10% - 71.55% | 49.58% - 73.04%|
| clean_markdown_and_emojis |66.25% - 72.99% | 48.55% - 74.01% | 70.34% - 71.69% | 56.23% - 73.95%|
| clean_text |99.03% - 87.10% | 95.97% - 88.13% | 98.28% - 86.12% | 97.89% - 87.81%|
| clean_numbers |99.03% - 87.10% | 95.97% - 88.13% | 98.28% - 86.12% | 97.89% - 87.81%|

In [253]:
def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re


mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'

                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]

    return mispellings_re.sub(replace, text)

In [254]:
# train.Question = train.Question.apply(replace_typical_misspell)
# test.Question = test.Question.apply(replace_typical_misspell)
# train.Response = train.Response.apply(replace_typical_misspell)
# test.Response = test.Response.apply(replace_typical_misspell)

In [255]:
!pip install -q pyspellchecker

In [256]:
from textblob import TextBlob
from spellchecker import SpellChecker
from multiprocessing import Pool
num_processes = 4
def correct_misspelling(original_text):
    spell = SpellChecker()
    words = original_text.split()
    corrected_words = [spell.correction(word) for word in words]
    try:
        corrected_text = ' '.join(corrected_words)
    
        return corrected_text
    except:
        return original_text

In [257]:
from tqdm import tqdm

it takes 18min.

In [260]:
%%time
with Pool(num_processes) as pool:
    test.Question = list(tqdm(pool.imap(correct_misspelling, test.Question) ,total=len(test.Question), desc="Processing test.Question"))
    test.Response = list(tqdm(pool.imap(correct_misspelling, test.Response),total=len(test.Response), desc="Processing test.Response"))
    train.Question = list(tqdm(pool.imap(correct_misspelling, train.Question),total=len(train.Question), desc="Processing train.Question"))
    train.Response = list(tqdm(pool.imap(correct_misspelling, train.Response),total=len(train.Response), desc="Processing train.Response"))

Processing: 100%|██████████| 3976/3976 [12:14<00:00,  5.41it/s]


CPU times: user 35.7 s, sys: 3.76 s, total: 39.5 s
Wall time: 18min 24s


In [261]:
oov = check_coverage(  build_vocab(train.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(train.Response.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Question.apply(lambda x: x.split())) ,embeddings_index)
oov = check_coverage(  build_vocab(test.Response.apply(lambda x: x.split())) ,embeddings_index)

100%|██████████| 1535/1535 [00:00<00:00, 245108.18it/s]


Found embeddings for 99.09% of vocab
Found embeddings for  87.10% of all text


100%|██████████| 23277/23277 [00:00<00:00, 356763.45it/s]


Found embeddings for 96.23% of vocab
Found embeddings for  88.17% of all text


100%|██████████| 633/633 [00:00<00:00, 330510.95it/s]


Found embeddings for 98.74% of vocab
Found embeddings for  86.33% of all text


100%|██████████| 11612/11612 [00:00<00:00, 380115.96it/s]

Found embeddings for 97.84% of vocab
Found embeddings for  87.81% of all text


|preprocess_fn| train q | train r | test q |test r|effect percent pre-step(test r)|
|-------|---------|---------|--------|------|------|
| google news emb first 5  |vocab  - all text | vocab  - all text | vocab  - all text| vocab  - all text|------|
| no fn|64.18% - 72.61% | 40.96% - 73.15% | 69.10% - 71.55% | 49.58% - 73.04%|------|
| clean_markdown_and_emojis |66.25% - 72.99% | 48.55% - 74.01% | 70.34% - 71.69% | 56.23% - 73.95%|13.41%- 1.2%|
| clean_text |99.03% - 87.10% | 95.97% - 88.13% | 98.28% - 86.12% | 97.89% - 87.81%|74.08% - 18.74%|
| clean_numbers |99.03% - 87.10% | 95.97% - 88.13% | 98.28% - 86.12% | 97.89% - 87.81%|0% - 0%|
| correct_misspelling |99.09% - 87.10% | 96.23% - 88.17% | 98.74% - 86.33% | 97.84% - 87.81%| 1% - -1%|
| before preprocess bert |51.25% - 69.74% | 20.14% - 71.52% | 57.37% - 70.21% | 30.73% - 71.50%| no_fns|
| fns result for bert |76.81% - 81.28% | 42.90% - 83.93% | 80.09% - 82.24% | 56.53% - 83.48%| all_fns|
<style>
th {
    background-color: #211fcc; /* preprocess_fn */
    text-align: center; /* Center-align text in header cells */
}
td:first-child {
    background-color: #211fcc; /* Background color for the first column (row indices) */
    text-align: center; /* Center-align text in the first column */
}
td {
    text-align: center; /* Center-align text in data cells */
}
</style>

In [264]:
import wandb

In [265]:
train

,Question,Response,target
0,Explain the concept of revolution,Coevolution is a biological process that occur...,3
1,Is it possible that recurring fever and chills...,Yes recurring fever and chills can be a sympto...,4
2,Evaluate the expression,The expression represents the factories of whi...,1
3,What are the roles of different types of ran i...,Messenger RNA mRNA mRNA carries genetic inf...,3
4,What is the role of gene flow in population ge...,Gene flow refers to the movement of individual...,3
...,...,...,...
3971,You will always find me in the past I can be c...,Please select one of the options from the tabl...,0
3972,To give me to someone i dont belong to is cowa...,points,0
3973,Could severe abdominal pain and vomiting be in...,Abdominal pain and vomiting can indeed be symp...,1
3974,How can I increase my productivity at work or ...,Prioritize tasks Make a to do list with the m...,1


In [266]:
test

,id,Question,Response
0,0,Multiply by,Multiplying by gives a result of Therefore the...
1,1,Who were the main protagonists in the Hundred ...,The most important characters in this war incl...
2,2,What was the importance of the Truman Doctrine,The Truman Doctrine also known as the United S...
3,3,Which film features a character named Jack wel...,The Nightmare Before Christmas
4,4,What are the components of the cytoskeleton,The cytoskeleton which provides the framework ...
...,...,...,...
996,996,Which film features a character named Jack wel...,Jack wellington is a main character in the Pus...
997,997,How does the size and scale of your model affe...,The size and scale of a language model can hav...
998,998,Can frequent urination and pelvic pain be symp...,Yes frequent urination and pelvic pain can be ...
999,999,Could sudden and severe chest pain be indicati...,Yes sudden and severe chest pain can be a symp...


In [267]:
run = wandb.init(
    project="h2o-ai-predict-the-llm-kaggle-competition", 
    entity=None, 
   job_type="preprocess"
)

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [268]:
raw_data_at = run.use_artifact('mustafakeser/h2o-ai-predict-the-llm-kaggle-competition/detect_llm_raw_data:v1', 
                               type='raw_data')
artifact_dir = raw_data_at.download()

wandb:   4 of 4 files downloaded.  


In [269]:
processed_data_at = wandb.Artifact("detect-llm-proprocessed-dataset", type="preprocess_data")

In [272]:
test

,id,Question,Response
0,0,Multiply by,Multiplying by gives a result of Therefore the...
1,1,Who were the main protagonists in the Hundred ...,The most important characters in this war incl...
2,2,What was the importance of the Truman Doctrine,The Truman Doctrine also known as the United S...
3,3,Which film features a character named Jack wel...,The Nightmare Before Christmas
4,4,What are the components of the cytoskeleton,The cytoskeleton which provides the framework ...
...,...,...,...
996,996,Which film features a character named Jack wel...,Jack wellington is a main character in the Pus...
997,997,How does the size and scale of your model affe...,The size and scale of a language model can hav...
998,998,Can frequent urination and pelvic pain be symp...,Yes frequent urination and pelvic pain can be ...
999,999,Could sudden and severe chest pain be indicati...,Yes sudden and severe chest pain can be a symp...


In [273]:

tbl_df_train = wandb.Table(data=train.reset_index())
tbl_df_test   = wandb.Table(data=test)
wandb.log({"train_preprocessed": tbl_df_train})
wandb.log({"test_preprocessed": tbl_df_test})
processed_data_at.add(tbl_df_train, "preprocessed_train")
processed_data_at.add(tbl_df_test, "preprocessed_test")

In [274]:
run.log_artifact(processed_data_at)

<Artifact detect-llm-proprocessed-dataset>

In [275]:
run.finish()

wandb: WARNING Source type is set to 'artifact' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
